<a href="https://colab.research.google.com/github/XTMay/python-data-science-course/blob/main/notebooks/Lec_14_Association_Rules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 关联规则分析（Association Rules）

### 1. 理论与概念

关联规则分析是一种在大型数据集中发现变量之间有趣关系的机器学习技术。最常见的应用场景是**购物篮分析（Market Basket Analysis）**，用于发现顾客购买商品时的关联性，例如购买啤酒的顾客很可能也会购买尿布（一个经典的案例）。除了购物篮分析，关联规则还可应用于：

- **推荐系统:** 根据用户的购买历史或浏览行为推荐相关商品。
- **异常检测:** 发现数据中不符合常规模式的关联。
- **网页点击流分析:** 分析用户在网站上的浏览路径，优化网站结构。
- **医疗诊断:** 分析病症之间的关联性，辅助医生诊断。

在关联规则分析中，有三个核心概念需要理解：支持度、置信度、提升度。

- **支持度 (Support):** 衡量项集 A 和 B 在总交易记录中同时出现的频率。高支持度意味着项集 A 和 B 一起出现的次数较多。
- **置信度 (Confidence):** 衡量在包含项集 A 的交易记录中，项集 B 也出现的频率。高置信度表示如果顾客购买了 A，那么他们很有可能也会购买 B。
- **提升度 (Lift):** 衡量项集 A 的出现对项集 B 的出现的提升程度。
    - 如果 Lift(A→B) = 1，表示 A 和 B 是独立的，购买 A 对购买 B 没有影响。
    - 如果 Lift(A→B) > 1，表示购买 A 会增加购买 B 的可能性，A 和 B 之间存在正相关关系。
    - 如果 Lift(A→B) < 1，表示购买 A 会降低购买 B 的可能性，A 和 B 之间存在负相关关系。

### 2. 公式与说明

- **支持度公式:**
  $$ \text{Support}(A \rightarrow B) = P(A \cup B) = \frac{\text{Number of transactions containing A and B}}{\text{Total number of transactions}} $$
  **说明:** 项集 A 和 B 在总交易记录中同时出现的比例。

- **置信度公式:**
  $$ \text{Confidence}(A \rightarrow B) = P(B|A) = \frac{\text{Support}(A \cup B)}{\text{Support}(A)} = \frac{\text{Number of transactions containing A and B}}{\text{Number of transactions containing A}} $$
  **说明:** 在包含项集 A 的交易记录中，包含项集 B 的比例。

- **提升度公式:**
  $$ \text{Lift}(A \rightarrow B) = \frac{\text{Confidence}(A \rightarrow B)}{\text{Support}(B)} = \frac{P(A \cup B)}{P(A)P(B)} $$
  **说明:** 置信度与项集 B 的支持度之比，用于衡量规则的强度以及 A 和 B 的相关性。

### 3. 示例数据

假设我们有一个小型超市的交易数据集，包含 5 条交易记录：

| 交易ID | 商品列表        |
| ------ | --------------- |
| 1      | 啤酒, 尿布, 牛奶 |
| 2      | 啤酒, 尿布     |
| 3      | 啤酒, 可乐     |
| 4      | 尿布, 牛奶     |
| 5      | 牛奶, 可乐     |


为了在 Python 中处理这些数据，我们可以将其转化为列表的列表，其中每个内部列表代表一笔交易中的商品：

```python
transactions = [
    ['啤酒', '尿布', '牛奶'],
    ['啤酒', '尿布'],
    ['啤酒', '可乐'],
    ['尿布', '牛奶'],
    ['牛奶', '可乐']
]
```

或者使用热编码（One-Hot Encoding）矩阵，其中每一行代表一笔交易，每一列代表一个商品，值为 1 表示该商品出现在该交易中，值为 0 表示未出现。

```python
import pandas as pd

# 创建一个包含所有商品的列表
all_items = sorted(list(set([item for sublist in transactions for item in sublist])))

# 创建一个空的 DataFrame
data_dict = {item: [0] * len(transactions) for item in all_items}
one_hot_df = pd.DataFrame(data_dict)

# 填充 DataFrame
for i, transaction in enumerate(transactions):
    for item in transaction:
        if item in all_items: # 确保商品在 all_items 中
             one_hot_df.loc[i, item] = 1

print(one_hot_df)
```

### 4. Python 示例代码

我们将使用 `mlxtend` 库来进行关联规则挖掘。首先需要安装该库：

In [5]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [8]:
!pip install -i https://pypi.org/simple mlxtend

接下来，使用 `mlxtend` 实现 Apriori 算法和关联规则挖掘：

In [6]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# 示例交易数据
transactions = [
    ['啤酒', '尿布', '牛奶'],
    ['啤酒', '尿布'],
    ['啤酒', '可乐'],
    ['尿布', '牛奶'],
    ['牛奶', '可乐']
]

# 使用 TransactionEncoder 进行热编码
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

# 使用 Apriori 算法查找频繁项集
# min_support 参数设定最小支持度阈值
frequent_itemsets = apriori(df, min_support=0.4, use_colnames=True)

print("频繁项集：")
print(frequent_itemsets)


频繁项集：
   support  itemsets
0      0.4      (可乐)
1      0.6      (啤酒)
2      0.6      (尿布)
3      0.6      (牛奶)
4      0.4  (尿布, 啤酒)
5      0.4  (尿布, 牛奶)


In [7]:
# 生成关联规则
# min_threshold 参数设定最小置信度阈值
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

print("\n关联规则：")
print(rules)


关联规则：
  antecedents consequents  antecedent support  consequent support  support  \
0        (尿布)        (啤酒)                 0.6                 0.6      0.4   
1        (啤酒)        (尿布)                 0.6                 0.6      0.4   
2        (尿布)        (牛奶)                 0.6                 0.6      0.4   
3        (牛奶)        (尿布)                 0.6                 0.6      0.4   

   confidence      lift  representativity  leverage  conviction  \
0    0.666667  1.111111               1.0      0.04         1.2   
1    0.666667  1.111111               1.0      0.04         1.2   
2    0.666667  1.111111               1.0      0.04         1.2   
3    0.666667  1.111111               1.0      0.04         1.2   

   zhangs_metric  jaccard  certainty  kulczynski  
0           0.25      0.5   0.166667    0.666667  
1           0.25      0.5   0.166667    0.666667  
2           0.25      0.5   0.166667    0.666667  
3           0.25      0.5   0.166667    0.666667  


上面的代码会输出频繁项集（满足最小支持度要求的项集）和关联规则（满足最小置信度要求的规则），并包含支持度、置信度、提升度等指标。

### 5. 案例分析

**案例：超市购物篮分析**

一个大型超市希望通过分析顾客的购物数据来优化商品布局和促销策略。通过关联规则分析，他们发现以下规则：

- `{面包, 牛奶} -> {鸡蛋}`: 支持度 0.15，置信度 0.7，提升度 2.1
- `{啤酒} -> {尿布}`: 支持度 0.2，置信度 0.65，提升度 1.8


**分析：**

- **规则 1:** 购买面包和牛奶的顾客有 70% 的概率也会购买鸡蛋。提升度 2.1 远大于 1，表明购买面包和牛奶会显著增加购买鸡蛋的可能性。
- **规则 2:** 购买啤酒的顾客有 65% 的概率也会购买尿布。提升度 1.8 大于 1，也存在正相关性。


**商业价值与应用方法：**

- **商品布局优化:** 将面包、牛奶和鸡蛋摆放在相近的位置，方便顾客同时购买。将啤酒和尿布也摆放在相近的位置。
- **捆绑销售与促销:** 可以尝试将面包、牛奶和鸡蛋进行捆绑销售，或对同时购买啤酒和尿布的顾客提供优惠。
- **库存管理:** 根据关联规则预测商品的共同需求，更准确地进行库存备货。
- **个性化推荐:** 在电商平台上，如果用户购买了面包和牛奶，可以向其推荐鸡蛋。


### 6. 课堂练习

**练习数据集:** 使用上面提供的 `transactions` 数据集。

1.  **挖掘频繁项集:** 使用 Apriori 算法，设置 `min_support=0.2`，找出所有的频繁项集。
2.  **筛选关联规则:** 从挖掘出的频繁项集中生成关联规则，筛选出置信度 (`confidence`) 大于等于 0.7 的规则。
3.  **分析高提升度规则:** 在筛选出的规则中，分析提升度 (`lift`) 大于 1 的规则，并尝试提出可能的业务建议。

In [1]:
# 课堂练习代码框架
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

transactions = [
    ['啤酒', '尿布', '牛奶'],
    ['啤酒', '尿布'],
    ['啤酒', '可乐'],
    ['尿布', '牛奶'],
    ['牛奶', '可乐']
]

# 数据准备
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

# 1. 挖掘频繁项集 (min_support=0.2)
frequent_itemsets_exercise = apriori(df, min_support=0.2, use_colnames=True)
display(frequent_itemsets_exercise)

,support,itemsets
0,0.4,(可乐)
1,0.6,(啤酒)
2,0.6,(尿布)
3,0.6,(牛奶)
4,0.2,"(可乐, 啤酒)"
5,0.2,"(可乐, 牛奶)"
6,0.4,"(尿布, 啤酒)"
7,0.2,"(啤酒, 牛奶)"
8,0.4,"(尿布, 牛奶)"
9,0.2,"(尿布, 啤酒, 牛奶)"


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [2]:
# 2. 生成并筛选置信度 >= 0.7 的关联规则
rules_exercise = association_rules(frequent_itemsets_exercise, metric="confidence", min_threshold=0.7)
print("\n课堂练习 - 置信度 >= 0.7 的关联规则:")
display(rules_exercise)


课堂练习 - 置信度 >= 0.7 的关联规则:


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,"(啤酒, 牛奶)",(尿布),0.2,0.6,0.2,1.0,1.666667,1.0,0.08,inf,0.5,0.333333,1.0,0.666667


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [3]:
# 3. 分析 lift > 1 的规则
display(rules_exercise[rules_exercise['lift'] > 1])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,"(啤酒, 牛奶)",(尿布),0.2,0.6,0.2,1.0,1.666667,1.0,0.08,inf,0.5,0.333333,1.0,0.666667


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

### 7. 课后练习

**数据集:**

| 交易ID | 商品列表                               |
| ------ | -------------------------------------- |
| 1      | 苹果, 香蕉, 橙子                      |
| 2      | 香蕉, 橙子, 草莓                      |
| 3      | 苹果, 葡萄                            |
| 4      | 橙子, 草莓                            |
| 5      | 苹果, 香蕉, 葡萄                      |
| 6      | 橙子, 草莓                            |
| 7      | 苹果, 香蕉, 橙子                      |
| 8      | 葡萄, 橙子                           |
| 9      | 苹果, 香蕉                            |
| 10     | 橙子, 草莓, 葡萄                      |
| 11     | 苹果, 橙子                            |
| 12     | 香蕉, 葡萄                            |
| 13     | 苹果, 草莓                            |
| 14     | 橙子, 葡萄                            |
| 15     | 苹果, 香蕉, 橙子, 草莓                 |

**作业要求:**

1.  将上述交易数据转换为 Python 可处理的格式 (列表的列表或热编码矩阵)。
2.  使用 `mlxtend` 库，设置 `min_support=0.2` 和 `min_confidence=0.5`，挖掘频繁项集和关联规则。
3.  分析生成的关联规则，特别关注提升度 (`lift`) 大于 1 的规则。
4.  撰写一份分析报告，内容包括：
    -   频繁项集列表
    -   关联规则列表 (包含支持度、置信度、提升度)
    -   对重要规则的商业解读，例如：
        -   这条规则意味着什么？
        -   可以如何应用于商品推荐、交叉销售或商品布局？
        -   这条规则可能存在的局限性或需要注意的地方。

```python
# 作业代码框架
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# 作业交易数据
assignment_transactions = [
    ['苹果', '香蕉', '橙子'],
    ['香蕉', '橙子', '草莓'],
    ['苹果', '葡萄'],
    ['橙子', '草莓'],
    ['苹果', '香蕉', '葡萄'],
    ['橙子', '草莓'],
    ['苹果', '香蕉', '橙子'],
    ['葡萄', '橙子'],
    ['苹果', '香蕉'],
    ['橙子', '草莓', '葡萄'],
    ['苹果', '橙子'],
    ['香蕉', '葡萄'],
    ['苹果', '草莓'],
    ['橙子', '葡萄'],
    ['苹果', '香蕉', '橙子', '草莓']
]

# 数据准备 (请学生自行完成)
# ...

# 挖掘频繁项集 (min_support=0.2) 和关联规则 (min_confidence=0.5) (请学生自行完成)
# ...

# 分析报告撰写 (请学生自行完成)
```